In [117]:
from pyhive import hive
from TCLIService.ttypes import TOperationState
import os
from os import listdir
from os.path import isfile, join
import ast
import json
import graph_tool.all as gt


In [135]:
def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def parse_settings(setting):
    prop, value = setting.split(' ')[1].split('=')
    if value.isnumeric():
        return prop, int(value)
    elif is_float(value):
        return prop, float(value)
    return prop, value

def apply(setting):
    prop, value = setting
    if isinstance(value, int):
        query = 'set %s=%d'%(prop, value)
    elif isinstance(value, float): 
        query = 'set %s=%f'%(prop, value)
    else:
        query = 'set %s=%s'%(prop, value)
    cursor.execute(query, async=False)
 

def process_tree(tree):
    if not tree: return ''
    for op in tree:
        if 'children' not in tree[op]: 
            return op.replace(' Operator', '')
        return op.replace(' Operator', '') + ' | ' + process_tree(tree[op]['children'])

# Parse the insert query
def parse_table_op_tree(table_tree, metadata):
    for o in table_tree:
        if o == 'TableScan':
            if 'table:' in table_tree[o]:
                metadata['in_table'] = table_tree[o]['table:']
        metadata['ops'].append(o.replace(' Operator', ''))
        if 'children' in table_tree[o]:
            parse_table_op_tree(table_tree[o]['children'], metadata)


def graph_annotation(annotations_tree):
    exec_plan = annotations_tree
   
    metadata = {}
    for stg_id in exec_plan:
        stg_plan = exec_plan[stg_id]
        for stg_op in stg_plan:
            op_works = stg_plan[stg_op]

            metadata[stg_id] = {'ops': []}
            if stg_op == "Stats-Aggr Operator":
                metadata[stg_id]['ops'].append(stg_op.replace(' Operator', ''))
            elif stg_op == "Move Operator":
                metadata[stg_id]['ops'].append(stg_op.replace(' Operator', ''))
                metadata[stg_id]['out_table'] =  op_works['tables:']['table:']["name:"]
            elif stg_op in ['Map Reduce', 'Map Reduce Local Work']:        
                for op in op_works:
                    if op in ["Alias -> Map Local Operator Tree:", "Alias -> Map Local Tables:"]:
                        op_tree = op_works[op]
                        for table_name in op_tree:
                            parse_table_op_tree(op_tree[table_name], metadata[stg_id])
                    elif op == "Map Operator Tree:":
                        op_tree = op_works[op][0]
                        parse_table_op_tree(op_tree, metadata[stg_id])
    return metadata


def build_graph_from_execution_plan(explain_cmd_json):
    execution_plan = explain_cmd_json['STAGE DEPENDENCIES']
    annotations_tree = graph_annotation(explain_cmd_json['STAGE PLANS'])

    label_vid_map = {}
    g = gt.Graph(directed=True)
    g.vertex_properties['label'] = g.new_vertex_property("string")
    g.vertex_properties['ops'] = g.new_vertex_property("vector<string>")
    g.vertex_properties['ext_input'] = g.new_vertex_property("string")
    
    for sid in execution_plan:
        if sid not in label_vid_map:
            vid = g.add_vertex()
            label_vid_map[sid] = vid
            g.vp.label[vid] = sid
            g.vp.ops[vid] = annotations_tree[sid]['ops']
            #print(annotations_tree[sid])
        
    for sid in execution_plan:
        if 'DEPENDENT STAGES' in execution_plan[sid]:
            dep_stages = [g.add_edge(label_vid_map[dsid], label_vid_map[sid]) for dsid in execution_plan[sid]['DEPENDENT STAGES'].replace(' ', '').split(',')]; dep_stages
        if 'CONDITIONAL CHILD TASKS' in execution_plan[sid]:
            dep_stages = [g.add_edge(label_vid_map[sid], label_vid_map[dsid]) for dsid in execution_plan[sid]['CONDITIONAL CHILD TASKS'].replace(' ', '').split(',')]; dep_stages
    return g


def main():
    cursor = hive.connect('neu-3-1').cursor() 

    # hive configuration file
    benchmark_path = '/local0/Kariz/expriments/benchmark/hive-testbench/sample-queries-tpcds'
    queries = ['%s/%s'%(benchmark_path, f) for f in listdir(benchmark_path) if (f.endswith(".sql") and isfile(join(benchmark_path, f)))]
    queries.sort()

    with open('%s/testbench.settings'%(benchmark_path), 'r') as fd:
        apply_settings = [apply(parse_settings(query)) for query in fd.read().split(';\n')[:-1]]; apply_settings

    query = 'use tpcds_bin_partitioned_textfile_10'
    cursor.execute(query, async=False)

    for index, query_file in enumerate(queries): 
        with open(query_file, 'r') as fd:
            if index == 46:
                continue;
            print(index, query_file.split('/')[-1].split('.')[0])
            query = 'EXPLAIN FORMATTED %s'%(fd.read().replace(';', ''))
            cursor.execute(query, async=False)
            res = cursor.fetchall()
            explain_cmd_json = json.loads(res[0][0])
            g = build_graph_from_execution_plan(explain_cmd_json)
            print('\t', g.num_vertices(), g.num_edges())
            gt.graph_draw(g, vertex_text=g.vertex_index, 
                          output='%s.png'%(query_file.split('/')[-1].split('.')[0]),
                          output_size=(1000, 1000))
main()

0 query12
	 11 12
1 query13
	 24 29
2 query15
	 19 23
3 query17
	 36 45
4 query18
	 37 47
5 query19
	 23 28
6 query20
	 11 12
7 query21
	 11 12
8 query22
	 17 20
9 query24
	 62 79
10 query25
	 36 45
11 query26
	 17 20
12 query27
	 18 21
13 query28
	 21 26
14 query29
	 36 45
15 query3
	 10 11
16 query31
	 86 110
17 query32
	 19 23
18 query34
	 18 21
19 query39
	 38 47
20 query40
	 18 21
21 query42
	 10 11
22 query43
	 11 12
23 query45
	 32 40
24 query46
	 27 33
25 query48
	 22 27
26 query49
	 32 37
27 query50
	 18 21
28 query51
	 14 15
29 query52
	 10 11
30 query54
	 36 44
31 query55
	 10 11
32 query56
	 66 83
33 query58
	 70 90
34 query60
	 66 83
35 query63
	 12 13
36 query64
	 172 219
37 query65
	 26 32
38 query66
	 23 26
39 query67
	 16 18
40 query68
	 31 38
41 query7
	 17 20
42 query70
	 28 33
43 query71
	 22 26
44 query72
	 38 46
45 query73
	 18 21
47 query76
	 24 29
48 query79
	 14 16
49 query80
	 54 65
50 query82
	 11 12
51 query83
	 67 84
52 query84
	 15 17
53 query85
	 31 38
54